In [ ]:
# сбор датасета осуществлялся 25 февраля 2025, все собранные вакансии актуальны на дату сбора

In [ ]:
import requests
import json
import pandas as pd

# подключение к hh.ru, определение параметров
url = 'https://api.hh.ru/vacancies'
params_list = [
    # Аналитик данных
    {'text': 'Аналитик данных', 'search_field': 'name', 'area': 1, 'per_page': 50, 'experience': 'noExperience', 'date_from': '2025-01-01T00:00:00', 'date_to': '2025-02-25T23:59:59'},
    {'text': 'Аналитик данных', 'search_field': 'name', 'area': 1, 'per_page': 50, 'experience': 'between1And3', 'date_from': '2025-01-01T00:00:00', 'date_to': '2025-02-25T23:59:59'},

    {'text': 'Аналитик данных', 'search_field': 'name', 'area': 2, 'per_page': 50, 'experience': 'noExperience', 'date_from': '2025-01-01T00:00:00', 'date_to': '2025-02-25T23:59:59'},
    {'text': 'Аналитик данных', 'search_field': 'name', 'area': 2, 'per_page': 50, 'experience': 'between1And3', 'date_from': '2025-01-01T00:00:00', 'date_to': '2025-02-25T23:59:59'},


    # Менеджер проектов
    {'text': 'Менеджер проектов', 'search_field': 'name', 'area': 1, 'per_page': 50, 'experience': 'noExperience', 'date_from': '2025-01-01T00:00:00', 'date_to': '2025-02-25T23:59:59'},
    {'text': 'Менеджер проектов', 'search_field': 'name', 'area': 1, 'per_page': 50, 'experience': 'between1And3', 'date_from': '2025-01-01T00:00:00', 'date_to': '2025-02-25T23:59:59'},

    {'text': 'Менеджер проектов', 'search_field': 'name', 'area': 2, 'per_page': 50, 'experience': 'noExperience', 'date_from': '2025-01-01T00:00:00', 'date_to': '2025-02-25T23:59:59'},
    {'text': 'Менеджер проектов', 'search_field': 'name', 'area': 2, 'per_page': 50, 'experience': 'between1And3', 'date_from': '2025-01-01T00:00:00', 'date_to': '2025-02-25T23:59:59'}
]

# рубли рубли в моей крови ----------------------------- АХВХХАВХ

def get_usd_to_rur():
    urll = "https://www.cbr-xml-daily.ru/daily_json.js"
    response = requests.get(urll)
    if response.status_code == 200:
        dataa = response.json()
        return dataa.get("Valute", {}).get("USD", {}).get("Value", 92.5)
    else:
        print(f"Ошибка получения курса валют: {response.status_code}") # Даша думает, что это убрать надо и я прогнала код без этой строки - все норм работает
        return 92.5

def convert_to_rur(salary, currency, rate):
    if salary and currency == "USD":
        return int(salary * rate)
    return salary

USD_TO_RUR = get_usd_to_rur()
#----------------------------

vacancies_list = []

for params in params_list:
    page = 0

    while True:
        params['page'] = page

        response = requests.get(url, params=params)

        data = response.json()
        vacancies = data.get('items', [])

        if not vacancies:
            break

        for vacancy in vacancies:
            description = vacancy.get('description') or ''
            requirement = vacancy.get('snippet', {}).get('requirement') or ''

            description = description.lower()
            requirement = requirement.lower()

            skills = []
            for skill in ['excel', 'bi', 'sql', 'python']:
              if skill in description or skill in requirement:
                skills.append(skill.upper())

            skills_str = ', '.join(skills) if skills else None

#все так хотят купить ----------

            salary_from = vacancy['salary']['from'] if vacancy.get('salary') else None
            salary_to = vacancy['salary']['to'] if vacancy.get('salary') else None
            currency = vacancy['salary']['currency'] if vacancy.get('salary') else None
            salary_from_rub = convert_to_rur(salary_from, currency, USD_TO_RUR)
            salary_to_rub = convert_to_rur(salary_to, currency, USD_TO_RUR)

            vacancies_list.append({
                'name': vacancy.get('name'),
                'area': vacancy['area']['name'],
                'employer': vacancy['employer']['name'] if vacancy.get('employer') else None,
                'salary_from': salary_from_rub,
                'salary_to': salary_to_rub,
                'currency': "RUR" if currency == "USD" else currency,
                'experience': vacancy['experience']['name'] if vacancy.get('experience') else None,
                'schedule': vacancy.get('schedule', {}).get('name') if vacancy.get('schedule') else None,
                'required_skills': skills_str
            })

        if page >= data.get('pages', 0) - 1:
            break

        page += 1

vacancies_df = pd.DataFrame(vacancies_list)

# удаление строк с пропущенными значениями в зп (сильно сокращает выборку)

vacancies_df.dropna(subset=['salary_from', 'salary_to'], inplace=True)

vacancies_df.reset_index(drop=True, inplace=True)

vacancies_df.to_excel("vacancies_ruri.xlsx", index=False)

vacancies_df

,name,area,employer,salary_from,salary_to,currency,experience,schedule,required_skills
0,Стажер аналитик данных ВТБ Юниор,Москва,Банк ВТБ (ПАО),75000.0,75000.0,RUR,Нет опыта,Полный день,"BI, SQL, PYTHON"
1,Менеджер проектов,Москва,Polza Agency (ИП Ерхов Никита Владимирович),50000.0,85000.0,RUR,Нет опыта,Удаленная работа,None
2,Менеджер проекта,Москва,Cинтера,70000.0,170000.0,RUR,Нет опыта,Полный день,None
3,Ассистент руководителя проекта,Москва,Технова,80000.0,120000.0,RUR,Нет опыта,Полный день,None
4,Помощник руководителя проектов,Москва,Ашманов и партнеры,40000.0,50000.0,RUR,Нет опыта,Полный день,None
5,Специалист по работе с документами/Помощник ме...,Москва,ДЖЕЙКЕТ РАБОТА,45000.0,65000.0,RUR,Нет опыта,Полный день,EXCEL
6,Junior Project manager (non IT),Москва,Fundmates,84640.0,101568.0,RUR,От 1 года до 3 лет,Удаленная работа,None
7,Проектный менеджер,Москва,Кеменова Мария Алексеевна,100000.0,300000.0,RUR,От 1 года до 3 лет,Удаленная работа,None
8,Менеджер образовательного проекта,Москва,Высшая школа бизнеса МГУ им. М. В. Ломоносова,60000.0,70000.0,RUR,От 1 года до 3 лет,Полный день,None
9,Менеджер по проектам / руководитель проектов,Москва,Форест,120000.0,400000.0,RUR,От 1 года до 3 лет,Полный день,None
